# Example 3 - Parameter Exploration

Due to the deterministic approach that [Tanaka *et al.*, 1999](http://linkinghub.elsevier.com/retrieve/pii/S0040195199000724) uses for estimating $Z_b$, the uncertainties returned by `CurieGrid.tanaka1999` are the simple results of error propagation from the initial spectra uncertainties. The spectra uncertainties represent the variance within the sampled sector of the 2D FFT, and may comprise both noise and anisotropic signal.

We present here several tests of the `CurieGrid.tanaka1999` function in order to explore how the input parameters affect the resulting Curie point depth estimates. In particular, we consider the following:

1. Effect of where in $k$-space is specific window used, separately for both the power and $k$-weighted power spectra (i.e., which $k$ do we consider?).
2. Effect of $k$-space window size (i.e., how much of the $k$-domain do we consider?). It is important to note that increasing the $k$-bandwidth also stabilises the estimation by increasing the statistics of the spectrum portion.
3. Effect of magnetic data window size (i.e., domain of magnetic data before computing FFT).

### Contents

- [Varying spatial frequency range](#Varying-spatial-frequency-range)
- [Varying spatial frequency size](#Varying-spatial-frequency-size)
- [Varying window size of magnetic anomaly](#Varying-window-size-of-magnetic-anomaly)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pycurious

In [ ]:
# load x,y,anomaly
mag_data = np.loadtxt("../../data/test_mag_data.txt")

nx, ny = 305, 305

x = mag_data[:,0]
y = mag_data[:,1]
d = mag_data[:,2].reshape(ny,nx)

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

# initialise object
grid = pycurious.CurieOptimiseTanaka(d, xmin, xmax, ymin, ymax)

# pick centroid
xpt = xmin + (xmax-xmin)/2
ypt = ymin + (ymax-ymin)/2

## Varying spatial frequency range

This test explores where in $k$-space is specific window used, separately for both the power and $k$-weighted power.

In [ ]:
# 1) Where in k-space do we locate window?
window_size = 304e3

zt_array = np.arange(0.0, 0.5, 0.05)
z0_array = np.arange(0.0, 0.5, 0.05)

z0q, ztq = np.meshgrid(z0_array, zt_array)

CPD_grid       = np.zeros((zt_array.size, z0_array.size))
sigma_CPD_grid = np.zeros((zt_array.size, z0_array.size))


for row, zt_min in enumerate(zt_array):
    for col, z0_min in enumerate(z0_array):
        zt_max = zt_min + 0.1
        z0_max = z0_min + 0.1

        zt_range = (zt_min, zt_max)
        z0_range = (z0_min, z0_max)

        zt, z0, zt_int, z0_int, sigma_zt, sigma_z0 = grid.optimise(window_size,
                                                                   xpt, 
                                                                   ypt,
                                                                   zt_range,
                                                                   z0_range,
                                                                   taper=None)

        CPD, sigma_CPD = grid.calculate_CPD(zt, z0, sigma_zt, sigma_z0)

        CPD_grid[row,col]       = CPD
        sigma_CPD_grid[row,col] = sigma_CPD


In [ ]:
fig = plt.figure(figsize=(12,10))
ax1 = fig.add_subplot(111, xlabel='z0', ylabel='zt')
sc1 = ax1.scatter(ztq.flat, z0q.flat, c=CPD_grid.flat, s=1000, marker='s')
fig.colorbar(sc1, label='CPD (km)')

## Varying spatial frequency size

Explore the effect of $k$-space window size (i.e., how much of the $k$-domain do we consider?). Increasing the $k$-bandwidth also stabilises the estimation by increasing the statistics of the spectrum portion.

In [ ]:
# 2) How big a bandwidth in k-space do we want?

nbins = 30
kspacing = np.linspace(0.05, 0.1, nbins)

CPD_grid       = np.zeros(nbins)
sigma_CPD_grid = np.zeros(nbins)

# minimum bounds for zt and z0
zt_min = 0.2
z0_min = 0.0

for i, dk in enumerate(kspacing):
    
    zt_max = zt_min + dk
    z0_max = z0_min + dk

    zt_range = (zt_min, zt_max)
    z0_range = (z0_min, z0_max)
    
    zt, z0, zt_int, z0_int, sigma_zt, sigma_z0 = grid.optimise(window_size,
                                                               xpt, 
                                                               ypt,
                                                               zt_range,
                                                               z0_range,
                                                               taper=None)
        
    CPD, sigma_CPD = grid.calculate_CPD(zt, z0, sigma_zt, sigma_z0)
    
    CPD_grid[i]       = CPD
    sigma_CPD_grid[i] = sigma_CPD

In [ ]:
fig2 = plt.figure(figsize=(8,8))
ax2 = fig2.add_subplot(111)
ax2.errorbar(kspacing, CPD_grid, yerr=sigma_CPD_grid, capsize=3)
ax2.invert_yaxis()
ax2.set_xlabel('Bandwidth (kmax - kmin)')
ax2.set_ylabel('CPD depth (km)')

The CPD estimates resulting from this test become consistent for $k$-bandwidths greater than approximately 0.1. Statistics improve with larger bandwidths, however, it is worth remaining aware of the typically non-linear trend of the spectrum, as it may not be appropriate for a linear fit over a large domain of $k$.

## Varying window size

Effect of magnetic data window size - i.e., domain of magnetic data before computing the FFT.

In [ ]:
# 3) How large a window width of magnetic data is required?

n_windows = 50
window_sizes = np.linspace(50e3, 304e3, n_windows)

CPD_grid       = np.zeros(n_windows)
sigma_CPD_grid = np.zeros(n_windows)

for i, window in enumerate(window_sizes):

    zt, z0, zt_int, z0_int, sigma_zt, sigma_z0 = grid.optimise(window,
                                                               xpt, 
                                                               ypt,
                                                               taper=None)

    CPD, sigma_CPD = grid.calculate_CPD(zt, z0, sigma_zt, sigma_z0)
    
    CPD_grid[i] = CPD
    sigma_CPD_grid[i] = sigma_CPD

In [ ]:
fig = plt.figure(figsize=(8,6))
ax1 = fig.add_subplot(111, xlabel='Window size (km)', ylabel='CPD (km)')
ax1.errorbar(window_sizes, CPD_grid, yerr=sigma_CPD_grid, capsize=3)

The results of this test highlight that there is a significant amount of variability in the CPD estimates at all window sizes, inferring that where possible, a range of window sizes should be tested for any data set. However, the righthand plot of propagated error in the CPD estimates clarifies that estimates from larger windows are more robust. In particular, with the prior knowledge that the CPD of the test data is 16 km, window sizes of greater than 80 km show a reduced improvement in precision. 